In [ ]:
# imports
import pandas as pd


In [8]:
# load datasets
train_df = pd.read_csv('/home/chris/Documents/Sentiment-Analysis/Datasets/twitter_training.csv')
val_df =pd.read_csv('/home/chris/Documents/Sentiment-Analysis/Datasets/twitter_validation.csv')

# Basic exploration
print("Training set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("\nTraining set info:")
print(train_df.info())
print("\nClass distribution in training:")
# First, let's see what columns you actually have
print("Column names in training set:")
print(train_df.columns.tolist())
print("\nFirst few rows of training set:")
print(train_df.head())

print("\nColumn names in validation set:")
print(val_df.columns.tolist())
print("\nFirst few rows of validation set:")
print(val_df.head())


Training set shape: (74681, 4)
Validation set shape: (999, 4)

Training set info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   2401                                                   74681 non-null  int64 
 1   Borderlands                                            74681 non-null  object
 2   Positive                                               74681 non-null  object
 3   im getting on borderlands and i will murder you all ,  73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB
None

Class distribution in training:
Column names in training set:
['2401', 'Borderlands', 'Positive', 'im getting on borderlands and i will murder you all ,']

First few rows of training set:
   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   


In [9]:
# Check for missing values
print("\nMissing values in training set:")
print(train_df.isnull().sum())

print("\nMissing values in validation set:")
print(val_df.isnull().sum())

# Check data types
print("\nTraining set dtypes:")
print(train_df.dtypes)


Missing values in training set:
2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    686
dtype: int64

Missing values in validation set:
3364                                                                                                                                                                                                                                                  0
Facebook                                                                                                                                                                                                                                              0
Irrelevant                                                                                                                                                                         